## Universidad Autonoma de Aguascalientes
## Departamento: Ciencias de la Computación
## Carrera: Ingeníeria en Computación Inteligente
### Curso: Metaheuristicas I
### Maestro: Dr. Francisco Javier Luna Rosas
### Alumnos: 
### Dante Alejandro Alegria Romero
### Diego Alberto Aranda Gonzalez
### Andrea Margarita Balandran Felix
### Diego Emilia Moreno Sanchez
### Semestre: Agosto_Diciembre del 2024"
### Examen 2do Parcial
# **El auto-reproducible no se ejecuta, lo tiene que descargar y ejecutar desde la cmd**
# **Ejemplo: python Examen2.py**

#### Importar librerias

In [3]:
import json
import time
import multiprocessing as mp 
import matplotlib.pyplot as plt
import numpy as np
import pyswarms as ps
import pandas as pd

#### Funciones para el manejo del JSON

In [4]:
def read_json(file_path='reviews.json'):
  with open(file_path) as file:
    return json.load(file)
  
def process_window(tasks):
  return sum(tasks['time'] for tasks in tasks)

#### Parametros

In [6]:
window_size = 100
num_processors = 8
max_reviews = 100000
tasks = read_json("reviews.json")

#### Procesamiento en Parallelo

In [ ]:
!python parallel_processing.py

In [8]:
def pso_objective(assignments, window, num_processors):
    assignments = np.round(assignments).astype(int)
    assignments = np.clip(assignments, 0, num_processors - 1)
    processor_times = np.zeros(num_processors)
    for task, processor in zip(window, assignments):
        processor_times[processor] += task['time']
    return np.max(processor_times)

def pso_assignment(window, num_processors):
    num_tasks = len(window)
    bounds = (np.zeros(num_tasks), np.ones(num_tasks) * (num_processors - 1))
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
    optimizer = ps.single.GlobalBestPSO(n_particles=60, dimensions=num_tasks, options=options, bounds=bounds)
    best_cost, best_pos = optimizer.optimize(lambda x: pso_objective(x, window, num_processors), iters=50)
    return best_cost

def pso_processing(tasks, window_size, num_processors):
    total_time = 0
    for i in range(0, len(tasks), window_size):
        window = tasks[i:i + window_size]
        makespan = pso_assignment(window, num_processors)
        total_time += makespan
    return total_time


In [1]:
pso_times = []
review_counts = list(range(10000, max_reviews + 1, 10000))
for count in review_counts:
    current_tasks = tasks[:count]
    start_time = time.time()
    pso_processing(current_tasks, window_size, num_processors)
    pso_times.append(time.time() - start_time)
    print(f"Processed {count} reviews")

NameError: name 'max_reviews' is not defined

#### Graficas y Tablas de los Resultados

In [ ]:
#Grafica de lineas
plt.figure(figsize=(12,8))
plt.plot(review_counts, parallel_times, label='Parallel Processing', marker='o')
plt.plot(review_counts, pso_times, label='Sequential Processing', marker='o')
plt.xlabel('Number of Reviews')
plt.ylabel('Processing Time (s)')
plt.title('Movie Review Processing Time')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,8))
bar_width = 0.35
index = np.arange(len(review_counts))
bar1 = ax1.bar(index, parallel_times, bar_width, label='Parallel Processing', color='b')
bar2 = ax1.bar(index + bar_width, pso_times, bar_width, label='Sequential Processing', color='r')
ax1.set_xlabel('Number of Reviews')
ax1.set_ylabel('Processing Time (s)')
ax1.set_title('Movie Review Processing Time')
ax1.set_xticks(index + bar_width / 2)
ax1.set_xticklabels(review_counts)
ax1.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#Tabla
speedup = [p / o for p, o in zip(parallel_times, pso_times)]
efficiency = [s / num_processors for s in speedup]
results_df = pd.DataFrame({
    'Reviews': review_counts,
    'Parallel Time': parallel_times,
    'PSO Time': pso_times,
    'Speedup': speedup,
    'Efficiency': efficiency
})
results_df.to_string(index=False)